In [2]:
import sys
sys.path.append('./src')
from config import *

In [3]:
import mPyPl as mp
import mPyPl as mp
import mPyPl.utils.image as mpui
from mpyplx import *
from pipe import Pipe
from moviepy.editor import *
import numpy as np
import itertools
import cv2
import math
import matplotlib.pyplot as plt
import keras

In [1]:
def close_clip(video):
    video.reader.close()
    video.audio.reader.close_proc()

In [34]:
classes = mp.get_classes(data_dir)
print(classes)

In [28]:
data = {}
for cls in [0,1]:
    data[cls] = (
       mp.get_datastream(data_dir,ext=".resized.mp4")
     | mp.filter('class_id',lambda x: x==cls)
     | mp.take(5)
     | mp.apply('filename','clip',lambda fn: VideoFileClip(fn).fx(vfx.resize,width=64))
     | mp.apply('clip','video',lambda c: np.array(list(c.iter_frames())))
     | mp.iter('clip',close_clip)
     | mp.select_field('video')
     | mp.as_list
    )

In [29]:
mpui.show_images([x[0] for x in data[0]])
mpui.show_images([x[0] for x in data[1]])

In [14]:
vgg = keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',input_shape=(36,64,3))

def get_vgg(video):
    res = vgg.predict(keras.applications.vgg16.preprocess_input(video))
    return res

In [30]:
vggs = {}
for cls in [0,1]:
    vggs[cls] = [get_vgg(x).reshape(127,-1) for x in data[cls]]

In [31]:
mpui.show_images(vggs[0],cols=5)
mpui.show_images(vggs[1],cols=5)